In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten,Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.backend import sigmoid
from sklearn.metrics import confusion_matrix
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from math import sqrt
from multiprocessing.pool import ThreadPool
import time
from sklearn.model_selection import GridSearchCV
import multiprocessing
import traceback
import tensorflow as tf
from sklearn.metrics import precision_score, make_scorer
from tensorflow.keras import backend as K


In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [ ]:
def reshape_data(x_train, x_test, y_train, y_test):
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
    y_test = np.array(y_test)
    y_test = np.reshape(y_test, (y_test.shape[0],1))

    return (x_train, x_test, y_train, y_test)

In [ ]:
def error_metrics(y_true, y_pred):
    rmse = sqrt(metrics.mean_squared_error(y_true, y_pred))
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse}

In [ ]:
def create_confusion_matrix(y_true,y_pred):
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [ ]:
def create_model(input_shape,optimizer='adam',activation='relu',dropout_rate=0.2,neurons=32,layers=3):
    model = Sequential()
    model.add(Conv1D(neurons, 1, activation=activation, input_shape=input_shape))
    model.add(Flatten())
    for i in range(layers):
        model.add(Dense(neurons, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation=activation))
    model.compile(loss='binary_crossentropy',optimizer=optimizer,)
    return model

In [ ]:
def run_process_model(optimizer = 'SGD',activation='relu',dropout_rate=0.2,neurons=64,layers=3,batch_size=64,epochs=25):
    print(optimizer,activation,dropout_rate,neurons,layers,batch_size,epochs)
    try:
        global x_train, x_test, y_train, y_test
        input_shape = (x_train.shape[1],1)
        model = create_model(input_shape,optimizer=optimizer,activation=activation,dropout_rate=float(dropout_rate),neurons=int(neurons),layers=int(layers))
        history = model.fit(x_train,y_train,epochs = int(epochs) ,batch_size = int(batch_size), validation_data = (x_test,y_test),shuffle = False,verbose=0)
        y_pred = model.predict(x_test)
        s = 0.01
        t = 0.5
        while True:
            pred = np.array(y_pred>t,int)
            res = confusion_matrix(y_test,pred)
            if res[1][1] >= 100 or t <= 0:
              break
            t = t - s
        result = {}
        result.update({"threshold":t})
        
        confusion = create_confusion_matrix(y_test,pred)
        result.update(confusion)
        result.update({'epochs':epochs,'batch_size':batch_size})    
        result.update({'optimizer':optimizer,'activation':activation,'dropout_rate':dropout_rate,'neurons':neurons,'layers':layers})
        return result
    except Exception as e:
        print(e)
        return None

In [ ]:
# myresults = pd.DataFrame()
# batch_size = [25,32,64,100]
# epochs = [25,50,75,100]
# optimizer = ['SGD','Adam']
# # learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
# # momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
# activation = ['relu','sigmoid']
# dropout_rate = [0,0.1, 0.2, 0.3]
# neurons = [32,64,128]
# layers = [2,3,4]
# combos = np.array(np.meshgrid(optimizer,activation,dropout_rate, neurons,layers,batch_size,epochs)).T.reshape(-1,7)

In [ ]:
def compute():    
    best_epochs,best_batch_size,best_layer,best_neurons,best_dropout_rate,best_activation,best_optimizer = 25,25,2,32,0.2,'sigmoid','adam'
    prev = 0
    for b in [25,32,50,75,100]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=0.2,neurons=32,layers=2,batch_size=b,epochs=25)
        if (res['precision'] > prev):
            prev = res['precision']
            best_batch_size = res['batch_size']
        else:
            break
            
    for e in [50,75,100]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=0.2,neurons=32,layers=2,batch_size=best_batch_size,epochs=e)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_epochs = res['epochs']
        else:
            break
            
    for l in [3,4]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=0.2,neurons=32,layers=l,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_layer = res['layers']
        else:
            break
            
    for n in [64]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=0.2,neurons=n,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_neurons = res['neurons']
        else:
            break
    
    for d in [0.1,0]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=d,neurons=best_neurons,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_dropout_rate = res['dropout_rate']
        else:
            break
    
    
    for a in ['relu']:
        res = run_process_model(optimizer = 'adam',activation=a,dropout_rate=best_dropout_rate,neurons=best_neurons,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_activation = res['activation']
        else:
            break

    for o in ['sgd']:
        res = run_process_model(optimizer = o,activation=best_activation,dropout_rate=best_dropout_rate,neurons=best_neurons,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_optimizer = res['optimizer']
        else:
            break
    print(best_epochs,best_batch_size,best_layer,best_neurons,best_dropout_rate,best_activation,best_optimizer)
    return (best_epochs,best_batch_size,best_layer,best_neurons,best_dropout_rate,best_activation,best_optimizer)

In [ ]:
security_codes = list()
for filename in os.listdir("../input/newdata/grstocks"):
    security_codes.append(filename[2:-4])
security_codes.sort()

In [ ]:
for name in security_codes:
    try:
        df = pd.read_csv("../input/newdata/grstocks/"+"gr"+str(name)+".csv")
        df = pre_process_data(df,60)
        df,column = dependent_column(df,"Next Day Close Price GR")
        myresult = list()
        for t in [0.001,0.002,0.003,0.004,0.005]:
            print(name,t)
            try:
                df["Target"] = df[column].apply(lambda x : 1 if x >= t else 0)
                X = df.drop(columns=["Target",column])
                Y = df["Target"]
                x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
                (x_train, x_test, y_train, y_test) = reshape_data(x_train, x_test, y_train, y_test)
                input_shape = (x_train.shape[1],1)
                best_epochs,best_batch_size,best_layer,best_neurons,best_dropout_rate,best_activation,best_optimizer = compute()
                result = run_process_model(optimizer = best_optimizer,activation=best_activation,dropout_rate=best_dropout_rate,neurons=best_neurons,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
                result.update({"rate_of_growth":t})
                myresult.append(result)
            except:
                continue
        pd.DataFrame(myresult).to_csv("cnn_"+str(name)+".csv",index=None)
    except:
        pass